# Stationbench Tutorial

This tutorial demonstrates how to use the stationbench repository to:
1. Preprocess weather forecast and ground truth data
2. Calculate verification metrics
3. Compare multiple forecasts and visualize results

This tutorial runs in a notebook environment. The same commands can be run in a terminal or script.

## Setup

First, complete the [setup guide](setup.md) then import the required packages.

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os
from datetime import datetime
import stationbench

## 1. Data Preprocessing

Stationbench expects forecast data and ground truth observations in Zarr format. Let's look at both datasets.

### Data format

The forecast data should be a Zarr dataset with the following structure:

```
<xarray.Dataset>
Dimensions:
  - time: Forecast initialization times
  - prediction_timedelta: Forecast lead times
  - latitude: Grid latitudes
  - longitude: Grid longitudes

Coordinates:
  - latitude: (latitude) float32, grid latitudes in degrees North
  - longitude: (longitude) float32, grid longitudes in degrees East  
  - prediction_timedelta: (prediction_timedelta) timedelta64[ns], forecast lead times
  - time: (time) datetime64[ns], initialization times

Data variables:
  - 10m_wind_speed: (time, prediction_timedelta, latitude, longitude) float32
  - 2m_temperature: (time, prediction_timedelta, latitude, longitude) float32
```

The wind speed and temperature data should be in m/s and °C respectively.

Let's analyze ensemble forecast data from ECMWF's Integrated Forecast System using the datasets provided by WeatherBench2. 
We'll evaluate benchmark performance on every first day of each quarter throughout 2022 at 12:00 UTC. 


In [2]:
forecast = xr.open_zarr("gs://weatherbench2/datasets/ifs_ens/2018-2022-1440x721_mean.zarr")
forecast = forecast[['10m_wind_speed', '2m_temperature']]

# select only the first day of each quarter
time_mask_forecast = (
    (forecast.time.dt.year == 2022) & 
    (forecast.time.dt.day == 1) & 
    (forecast.time.dt.month.isin([1, 4, 7, 10])) & 
    (forecast.time.dt.hour == 12)
)
forecast = forecast.sel(time=forecast.time[time_mask_forecast])

# select only lead times 0 to 10 days every 24 hours (the data is 6 hourly -> step of 4|)
forecast = forecast.isel(prediction_timedelta=slice(0, 41, 4))  

forecast


<xarray.Dataset> Size: 365MB
Dimensions:               (time: 4, prediction_timedelta: 11, latitude: 721,
                           longitude: 1440)
Coordinates:
  * latitude              (latitude) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * longitude             (longitude) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 88B 0 days ....
  * time                  (time) datetime64[ns] 32B 2022-01-01T12:00:00 ... 2...
Data variables:
    10m_wind_speed        (time, prediction_timedelta, latitude, longitude) float32 183MB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    2m_temperature        (time, prediction_timedelta, latitude, longitude) float32 183MB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>

Let's also have a look at the METEOSTAT ground truth data.

In [3]:
stations = xr.open_zarr( "https://opendata.jua.sh/stationbench/meteostat_benchmark.zarr")
stations


<xarray.Dataset> Size: 7GB
Dimensions:         (time: 61368, station_id: 14491)
Coordinates:
    elevation       (station_id) int64 116kB dask.array<chunksize=(14491,), meta=np.ndarray>
    latitude        (station_id) float32 58kB dask.array<chunksize=(14491,), meta=np.ndarray>
    longitude       (station_id) float32 58kB dask.array<chunksize=(14491,), meta=np.ndarray>
  * station_id      (station_id) <U5 290kB '00FAY' '00TG6' ... 'ENHA0' 'ENHE0'
  * time            (time) datetime64[ns] 491kB 2018-01-01 ... 2024-12-31T23:...
Data variables:
    10m_wind_speed  (time, station_id) float32 4GB dask.array<chunksize=(4380, 2000), meta=np.ndarray>
    2m_temperature  (time, station_id) float32 4GB dask.array<chunksize=(4380, 2000), meta=np.ndarray>
Attributes:
    conventions:  CF-1.8
    created:      2025-01-09T18:24:30.010932+00:00
    created_by:   meteostat_csv_to_zarr.py
    crs:          EPSG:4326
    description:  Historical weather data from weather stations worldwide
    featureType:  timeSeries
    history:      Created at 2025-01-09T18:24:30.010873+00:00
    institution:  Meteostat
    references:   https://dev.meteostat.net/
    source:       https://bulk.meteostat.net/v2/
    title:        Meteostat Hourly Weather Data

Notice the difference that the ground truth data is not a grid but unstructured point data made up of stations. This package will automatically align the grid data to the station locations using linear interpolation.

## 2. Calculate Verification Metrics

Now we'll calculate RMSE between the forecast and ground truth data.
For this we need to set the following parameters:
- `--forecast`: Path or Xarray Dataset of forecast data (required)
- `--stations`: Path or Xarray Dataset of ground truth data (optional, defaults to METEOSTAT)
- `--start_date`: Start date for benchmarking (required)
- `--end_date`: End date for benchmarking (required)
- `--output`: Output path for benchmarks (required)
- `--region`: Region to benchmark (see `regions.py` for available regions)
- `--name_10m_wind_speed`: Name of 10m wind speed variable (optional)
- `--name_2m_temperature`: Name of 2m temperature variable (optional)

In [4]:
start_date = "2022-01-01"
end_date = "2022-12-31"
output_forecast = "data/forecast_benchmark.zarr"
region = "global"
name_10m_wind_speed = "10m_wind_speed"
name_2m_temperature = "2m_temperature"

stationbench.calculate_metrics(
    forecast=forecast,
    stations=stations,
    start_date=start_date,
    end_date=end_date,
    output=output_forecast,
    region=region,
    name_10m_wind_speed=name_10m_wind_speed,
    name_2m_temperature=name_2m_temperature,
    use_dask=True
)

2025-01-17 15:41:56,766 - stationbench.calculate_metrics - INFO - Dask dashboard http://127.0.0.1:8787/status
2025-01-17 15:41:56,767 - stationbench.calculate_metrics - INFO - Preparing stations data
2025-01-17 15:41:56,768 - stationbench.calculate_metrics - INFO - Selecting region: https://linestrings.com/bbox/#-180,-90,180,90
2025-01-17 15:41:57,536 - stationbench.calculate_metrics - INFO - Filtered stations: 14491 -> 14491
2025-01-17 15:41:57,538 - stationbench.calculate_metrics - INFO - Preparing forecast dataset
2025-01-17 15:41:57,544 - stationbench.calculate_metrics - INFO - Selecting region: https://linestrings.com/bbox/#-180,-90,180,90
2025-01-17 15:41:57,547 - stationbench.calculate_metrics - INFO - Converting longitudes from 0-360 to -180-180 range
2025-01-17 15:41:57,558 - stationbench.calculate_metrics - INFO - Renaming wind speed variable from 10m_wind_speed to 10m_wind_speed
2025-01-17 15:41:57,559 - stationbench.calculate_metrics - INFO - Renaming temperature variable f

<xarray.Dataset> Size: 2MB
Dimensions:         (lead_time: 11, station_id: 14491)
Coordinates:
  * lead_time       (lead_time) timedelta64[ns] 88B 0 days 1 days ... 10 days
    latitude        (station_id) float32 58kB 53.19 54.72 70.93 ... 60.82 65.33
    longitude       (station_id) float32 58kB -112.2 -113.3 ... 11.07 2.33
    elevation       (station_id) int64 116kB 688 515 10 0 10 ... 10 119 223 80
  * station_id      (station_id) <U5 290kB '00FAY' '00TG6' ... 'ENHA0' 'ENHE0'
Data variables:
    10m_wind_speed  (lead_time, station_id) float32 638kB 1.416 0.7709 ... 6.698
    2m_temperature  (lead_time, station_id) float32 638kB 2.262 2.052 ... 1.114

## 3. Compare Multiple Forecasts

For comparing the forecast against multiple reference forecasts, we need to set the following parameters:
- `--evaluation_benchmarks_loc`: Path to the evaluation benchmarks (required)
- `--reference_benchmark_locs`: Dictionary of reference benchmark locations, the first one is used for skill score (required)
- `--run_name`: W&B run name (required)
- `--regions`: Comma-separated list of regions, see `regions.py` for available regions (required)

Let's use as reference forecast, the ECMWF's HRES dataset, and also calculate the metrics for this dataset.

In [5]:
reference = xr.open_zarr("gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr")

# select only the first day of each quarter
time_mask_reference = (
    (reference.time.dt.year == 2022) & 
    (reference.time.dt.day == 1) & 
    (reference.time.dt.month.isin([1, 4, 7, 10])) & 
    (reference.time.dt.hour == 12)
)
reference = reference.sel(time=reference.time[time_mask_reference])


# select only lead times 0 to 10 days every 24 hours
reference = reference.isel(prediction_timedelta=slice(0, 41, 4))  

reference = reference[['10m_wind_speed', '2m_temperature']]
reference

<xarray.Dataset> Size: 365MB
Dimensions:               (time: 4, prediction_timedelta: 11, latitude: 721,
                           longitude: 1440)
Coordinates:
  * latitude              (latitude) float32 3kB -90.0 -89.75 ... 89.75 90.0
  * longitude             (longitude) float32 6kB 0.0 0.25 0.5 ... 359.5 359.8
  * prediction_timedelta  (prediction_timedelta) timedelta64[ns] 88B 0 days ....
  * time                  (time) datetime64[ns] 32B 2022-01-01T12:00:00 ... 2...
Data variables:
    10m_wind_speed        (time, prediction_timedelta, latitude, longitude) float32 183MB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>
    2m_temperature        (time, prediction_timedelta, latitude, longitude) float32 183MB dask.array<chunksize=(1, 1, 721, 1440), meta=np.ndarray>

In [6]:
output_reference = "data/reference_benchmark.zarr"

stationbench.calculate_metrics(
    forecast=reference,
    stations=stations,
    start_date=start_date,
    end_date=end_date,
    output=output_reference,
    region=region,
    name_10m_wind_speed=name_10m_wind_speed,
    name_2m_temperature=name_2m_temperature,
    use_dask=True
)

/home/andi/.cache/pypoetry/virtualenvs/stationbench-6RBOhfLp-py3.11/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41639 instead
  warnings.warn(


2025-01-17 15:42:07,570 - stationbench.calculate_metrics - INFO - Dask dashboard http://127.0.0.1:41639/status
2025-01-17 15:42:07,571 - stationbench.calculate_metrics - INFO - Preparing stations data
2025-01-17 15:42:07,572 - stationbench.calculate_metrics - INFO - Selecting region: https://linestrings.com/bbox/#-180,-90,180,90
2025-01-17 15:42:08,391 - stationbench.calculate_metrics - INFO - Filtered stations: 14491 -> 14491
2025-01-17 15:42:08,392 - stationbench.calculate_metrics - INFO - Preparing forecast dataset
2025-01-17 15:42:08,398 - stationbench.calculate_metrics - INFO - Selecting region: https://linestrings.com/bbox/#-180,-90,180,90
2025-01-17 15:42:08,400 - stationbench.calculate_metrics - INFO - Converting longitudes from 0-360 to -180-180 range
2025-01-17 15:42:08,414 - stationbench.calculate_metrics - INFO - Renaming wind speed variable from 10m_wind_speed to 10m_wind_speed
2025-01-17 15:42:08,416 - stationbench.calculate_metrics - INFO - Renaming temperature variable 

<xarray.Dataset> Size: 2MB
Dimensions:         (lead_time: 11, station_id: 14491)
Coordinates:
  * lead_time       (lead_time) timedelta64[ns] 88B 0 days 1 days ... 10 days
    latitude        (station_id) float32 58kB 53.19 54.72 70.93 ... 60.82 65.33
    longitude       (station_id) float32 58kB -112.2 -113.3 ... 11.07 2.33
    elevation       (station_id) int64 116kB 688 515 10 0 10 ... 10 119 223 80
  * station_id      (station_id) <U5 290kB '00FAY' '00TG6' ... 'ENHA0' 'ENHE0'
Data variables:
    10m_wind_speed  (lead_time, station_id) float32 638kB 1.437 0.8291 ... 9.353
    2m_temperature  (lead_time, station_id) float32 638kB 1.574 1.566 ... 1.716

Let's compare our forecast against the reference forecast and visualize the results using Weights & Biases.

In [8]:
evaluation_benchmarks_loc = "data/forecast_benchmark.zarr"
reference_benchmark_locs = {"Reference": "data/reference_benchmark.zarr"}
# include day of today in the run name
run_name = f"tutorial-comparison-{datetime.now().strftime('%Y-%m-%d')}"
regions = "global"

stationbench.compare_forecasts(
    evaluation_benchmarks_loc=evaluation_benchmarks_loc,
    reference_benchmark_locs=reference_benchmark_locs,
    run_name=run_name,
    regions=regions
)

2025-01-17 15:43:14,999 - stationbench.compare_forecasts - INFO - Logging metrics to WandB: https://wandb.ai/jua/stationbench/runs/tutorial-comparison_2025-01-17


2025-01-17 15:43:15,586 - stationbench.compare_forecasts - INFO - Incumbent artifact tutorial-comparison_2025-01-17-temporal_plots found...
2025-01-17 15:43:15,607 - stationbench.compare_forecasts - INFO - Point based benchmarks computed, generating plots and writing to wandb...
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  
wandb:   2 of 2 files downloaded.  


## Understanding the Results

The comparison generates several visualizations:

1. **Geographical scatter plots**:
   - RMSE values at each station location
   - Skill scores comparing against reference forecasts

2. **Time series plots**:
   - RMSE evolution over forecast lead time
   - Skill score evolution over forecast lead time

These plots are automatically uploaded to your W&B project where you can:
- Compare different model versions
- Track performance improvements
- Share results with your team

Visit your W&B project page to explore the interactive visualizations!